# Train U-Net

This notebook will guide you through the steps to train your unet-tracker model on Google colab.

With approximately 200-400 images in the training set, I trained for 200 epochs and got reasonable results for face tracking.

## GPU access

You will need to get a Runtime with GPU access. Click on Runtime/Change runtime type. Select GPU as hardware accelerator.

## Install unet-tracker

We need to install the unet-tracker python package into your Colab workspace.

In [ ]:
!pip install albumentations==1.3.0
!git clone https://github.com/kevin-allen/unetTracker
!pip install -r unetTracker/requirements.txt
!pip install -e unetTracker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.3.1
    Uninstalling albumentations-1.3.1:
      Successfully uninstalled albumentations-1.3.1
Cloning into 'unetTracker'...
remote: Enumerating objects: 941, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 941 (delta 194), reused 309 (delta 188), pack-reused 621
Receiving objects: 100% (941/941), 123.86 MiB | 24.90 MiB/s, done.
Resolving deltas: 100% (584/584), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.0/380.0 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 8.6

Obtaining file:///content/unetTracker
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for unetTracker (pyproject.toml) ... done
  Created wheel for unetTracker: filename=unetTracker-0.0.1-0.editable-py3-none-any.whl size=16048 sha256=0f73c93ea5a7f2da37ad5d780eb085d5626c2122a4de2b8fb5cbf80d2529d74b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2g7kmvvz/wheels/62/9b/5a/0cb547490a9187d698861d98e1e803c5e64f31a9d899a8e84c
Successfully built unetTracker


You now need to restart your runtime to be able to use unet-tracker. You can press the button above.

We are now going to mount your google drive so that we can access the unet-tracker project directory. Running the code below should open a window in your browser and you will need to give permission to Google Colab to access Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
fn = "/content/drive/My Drive/dsfolder"
if os.path.exists(fn):
  print("We can access the dsfolder directory.")
else:
  raise IOError("Problem accessing the dsfolder directory.")

We can access the dsfolder directory.


In [ ]:
# this will run the code in the setup_project.py and create a variable called `project`
fn = "/content/drive/My Drive/dsfolder/setup_project.py"
if os.path.exists(fn):
  print("We can access the file.")
else:
  raise IOError("Problem accessing the file.")

%run "/content/drive/My Drive/dsfolder/setup_project.py"

We can access the file.
Project directory: /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker
Getting configuration from config file. Values from config file will be used.
Loading /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/config.yalm
{'augmentation_HorizontalFlipProb': 0.5, 'augmentation_RandomBrightnessContrastProb': 0.2, 'augmentation_RandomSizedCropProb': 1.0, 'augmentation_RotateProb': 0.3, 'image_extension': '.png', 'image_size': [270, 480], 'labeling_ImageEnlargeFactor': 2.0, 'name': 'finger_tracker', 'normalization_values': {'means': [0.40666553378105164, 0.45300230383872986, 0.5148566961288452], 'stds': [0.23905879259109497, 0.2506697177886963, 0.2679229974746704]}, 'object_colors': [(0.0, 0.0, 255.0), (255.0, 0.0, 0.0), (255.0, 255.0, 0.0), (240.0, 255.0, 255.0)], 'objects': ['f1', 'f2', 'f3', 'f4'], 'target_radius': 6, 'unet_features': [64, 128, 256, 512]}


In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from torch import optim
import torch.nn as nn
from datetime import datetime
import albumentations as A
import cv2
import os
import pickle

from unetTracker.trackingProject import TrackingProject
from unetTracker.dataset import UNetDataset
from unetTracker.unet import Unet
from unetTracker.coordinatesFromSegmentationMask import CoordinatesFromSegmentationMask
from unetTracker.utils import check_accuracy

We can check if torch has access to a GPU.

In [ ]:
torch.cuda.is_available(),torch.cuda.get_device_name(0)

(True, 'Tesla T4')

## Hyperparameters

You can probably leave most of these parameters as they are.

If it is the first time you train your model, you can set `LOAD_MODEL` to False. If you only want to refine your model quickly, you can set it to True.

As a starting point, you can use ~100 epochs if you have between 400 to 1000 images.

In [ ]:
LEARNING_RATE=1e-4
DEVICE = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
BATCH_SIZE=4
NUM_EPOCHS = 100
NUM_WORKERS = 2
OUTPUT_CHANNELS = len(project.object_list)
IMAGE_HEIGHT = project.image_size[0]
IMAGE_WIDTH = project.image_size[1]
PIN_MEMORY = True
LOAD_MODEL = True # If this is the first time you train the model, set LOAD_MODEL to False
TRAIN_IMAGE_DIR = os.path.join(project.dataset_dir,"train_images")
TRAIN_MASK_DIR =  os.path.join(project.dataset_dir,"train_masks")
TRAIN_COORDINATE_DIR = os.path.join(project.dataset_dir,"train_coordinates")
VAL_IMAGE_DIR = os.path.join(project.dataset_dir,"val_images")
VAL_MASK_DIR =  os.path.join(project.dataset_dir,"val_masks")
VAL_COORDINATE_DIR = os.path.join(project.dataset_dir,"val_coordinates")

## Model, loss, and optimizer

In [ ]:
model = Unet(in_channels=3, out_channels=OUTPUT_CHANNELS).to(DEVICE)
if LOAD_MODEL:
    project.load_model(model)

# set the model in train mode
model.train()

loss_fn = nn.BCEWithLogitsLoss() # not doing sigmoid on the output of the model, so use this, if we had more classes (objects) we would use change out_chan and cross_entropy_loss as loss_fn
optimizer= optim.Adam(model.parameters(),lr=LEARNING_RATE)
scaler = torch.cuda.amp.GradScaler()

## Data augmentation and normalization



In [ ]:
fileName = os.path.join(project.augmentation_dir,"trainTransform")
print("Loading trainTransform from", fileName)
trainTransform=pickle.load(open(fileName,"rb" ))

fileName = os.path.join(project.augmentation_dir,"valTransform")
print("Loading valTransform from", fileName)
valTransform=pickle.load(open(fileName, "rb" ))

Loading trainTransform from /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/augmentation/trainTransform
Loading valTransform from /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/augmentation/valTransform


## Datasets and DataLoaders

In [ ]:
trainDataset = UNetDataset(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR,TRAIN_COORDINATE_DIR, transform=trainTransform)
valDataset = UNetDataset(VAL_IMAGE_DIR, VAL_MASK_DIR,VAL_COORDINATE_DIR, transform=valTransform)
trainLoader = DataLoader(trainDataset,shuffle=True,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory=PIN_MEMORY)
valLoader = DataLoader(valDataset,shuffle=False,batch_size=BATCH_SIZE, num_workers=NUM_WORKERS,pin_memory = PIN_MEMORY)

In [ ]:
imgs, masks, _ = next(iter(trainLoader))
imgs.shape, masks.shape

(torch.Size([4, 3, 270, 480]), torch.Size([4, 4, 270, 480]))

There is a lot of black because half of our pixels are below 0, on average.


# Save and load checkpoint

In [ ]:
def save_checkpoint(state, filename = "my_checkpoint.pth.tar"):
    #print("Saving checkpoint")
    torch.save(state,filename)

## Training loop

This is where we train our model. Every few epochs, the performance of the model will be evaluated on the validation set.

The task of the model is to learn to predict your masks. For each image in the dataset, you created a set of masks when you labelled the image. For one image, there is one mask per body part. The mask has the same size as your image.  Most pixels in the mask are set to 0, but the circle centered on the body part is set to 1.

The task of the model is to output values for each mask that ressemble the mask you created while labeling the images. The loss function just measure the similarity between your masks and the output of the model.

It ususally takes a 40-60 epochs before the model starts to mark the labeled positive pixels of your mask as positive pixels.

During training, you should have a look at the printed output. If your model is learning, the loss will decrease over time.

In [ ]:
def train_fn(loader,model,optimizer,loss_fn,scaler,epoch,total_epochs):
    """
    One epoch of training
    """
    loop = tqdm(loader)
    for batch_idx, (data,targets,_) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions,targets)


        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix_str("loss: {:.7f}, epoch: {:d}/{:d}".format(loss.item(),epoch,total_epochs))


In [ ]:
startTime = datetime.now()
print("Starting time:",startTime)
for epoch in range(NUM_EPOCHS):

    train_fn(trainLoader,model,optimizer,loss_fn,scaler,epoch,NUM_EPOCHS)

    if epoch % 5 == 0 :
        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict()}
        save_checkpoint(checkpoint,filename=os.path.join(project.models_dir,"my_checkpoint.pth.tar"))

        # check accuracy
        print("Performance on validation set")
        check_accuracy(model,valLoader,DEVICE)

endTime=datetime.now()
print("End time:",endTime)
print("{} epochs, duration:".format(NUM_EPOCHS), endTime-startTime)

Starting time: 2023-12-02 14:02:58.000685


  0%|          | 0/40 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 40/40 [00:49<00:00,  1.23s/it, loss: 0.0032969, epoch: 0/100]


Performance on validation set
Number of positive pixels predicted: 5596.0
Number of positive pixels in masks: 7739.0
Percentage of positive pixels predicted: 0.060
Percentage of positive pixels in masks: 0.083
Accuracy: 99.937
Dice score: 0.561
Mask pixels detected (True positives): 47.926%
False positives: 33.721%
Mean distance: 4.681390950497423


100%|██████████| 40/40 [00:08<00:00,  4.52it/s, loss: 0.0026228, epoch: 5/100]


Performance on validation set
Number of positive pixels predicted: 5897.0
Number of positive pixels in masks: 7739.0
Percentage of positive pixels predicted: 0.063
Percentage of positive pixels in masks: 0.083
Accuracy: 99.930
Dice score: 0.527
Mask pixels detected (True positives): 45.678%
False positives: 40.054%
Mean distance: 7.932054130557269


...

Performance on validation set
Number of positive pixels predicted: 5862.0
Number of positive pixels in masks: 7739.0
Percentage of positive pixels predicted: 0.063
Percentage of positive pixels in masks: 0.083
Accuracy: 99.933
Dice score: 0.546
Mask pixels detected (True positives): 47.732%
False positives: 36.984%
Mean distance: 4.874509634896315


100%|██████████| 40/40 [00:08<00:00,  4.50it/s, loss: 0.0006524, epoch: 95/100]


Performance on validation set
Number of positive pixels predicted: 7044.0
Number of positive pixels in masks: 7739.0
Percentage of positive pixels predicted: 0.075
Percentage of positive pixels in masks: 0.083
Accuracy: 99.931
Dice score: 0.574
Mask pixels detected (True positives): 54.012%
False positives: 40.659%
Mean distance: 3.6924108859075844


100%|██████████| 40/40 [00:08<00:00,  4.49it/s, loss: 0.0009320, epoch: 99/100]

End time: 2023-12-02 14:19:43.313190
100 epochs, duration: 0:16:45.312505


Once your model starts to predict positive pixels, the output will contain the a few values that helps you track how your model is doing.

* Dice score: a common measure to measure the performace of image segmentation models. [External link](https://medium.com/mlearning-ai/understanding-evaluation-metrics-in-medical-image-segmentation-d289a373a3f)
* Mask pixels detected: percentage of the positive pixels in the masks that are predicted as positive by the model.
* False positive: percentage of the pixels that are predicted as positive by the models that are negative pixels in your masks.
* Mean distance: Mean distance in pixels between the coordinates you labeled and the coordinated calculated from the model output.



In [ ]:
project.save_model(model)

saving model state dict to /content/drive/My Drive/dsfolder/trackingProjects/finger_tracker/models/UNet.pt
2023-12-02 13:05:27.481250
